In [1]:
# Gerekli olan kütüpanelerin import edilmesi.

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [3]:
# Veri setinin okunmasını sağlıyorum.
df = pd.read_csv(r"C:\Users\realb\Desktop\ML_dersler\ML_301\XGBoost\Hitters.csv")

# Veri setinin içinden eksik gözlemleri çıkarmak gerekiyor bu işlemi yapıyorum.
df = df.dropna()

# Şimdi ise dummies ile kukla veri işlemi yapıyorum. One end coding yöntemi ile bu işlemi yapıyorum.
dms = pd.get_dummies(df[['League', 'Division','NewLeague']])

# Bağımlı değişkenimi "y" atıyorum.
y = df["Salary"]

# Veri setimiz içindeki bağımlı değişkeni ve katogorik değişkenlerin götürülmesi işlemini yapıyorum.
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Sonuçunda yukardaki değerlerin dummi versyonlarını ve bağımsız değişkenlerimi bir arada tutma işlemini yapıyorum.
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

# Şimdi ise veri setimizde train ve test ayrımı yapmak için bu ayrımı yapma işlemini yapıyorum.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
# Fonksyon herhangi bir şekilde python yada jupiterin içinde gelmediği için install ediyorum.
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 330.3 kB/s eta 0:05:02
   ---------------------------------------- 0.1/99.8 MB 469.7 kB/s eta 0:03:33
   ---------------------------------------- 0.1/99.8 MB 778.5 kB/s eta 0:02:08
   ---------------------------------------- 0.3/99.8 MB 1.1 MB/s eta 0:01:29
   ---------------------------------------- 0.4/99.8 MB 1.6 MB/s eta 0:01:03
   ---------------------------------------- 0.6/99.8 MB 1.9 MB/s eta 0:00:54
   ---------------------------------------- 0.8/99.8 MB 2.2 MB/s eta 0:00:46
   ---------------------------------------- 1.0/99.8 MB 2.4 MB/s eta 0:00:42
    -----------------------------

In [5]:
# install etiğim fonksyonu buraya import ediyorum.
import xgboost

In [6]:
from xgboost import XGBRFRegressor

In [7]:
xgb = XGBRFRegressor().fit(X_train, y_train)

In [8]:
xgb

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=None, ...)

In [11]:
y_pred = xgb.predict(X_test)

In [12]:
np.sqrt(mean_squared_error(y_test, y_pred))

349.5198027640781

In [13]:
# Şimidi belli paremetrelerden bir sözlük yapısı oluşturup onun üzerinden bir çalışma yapacağım.
xbg_params = {"learning_rate": [0.1,0.01,0.5],         # Aşırı öğrenmenin önüne geçmek için yapılan bir çalışmadır.
             "max_depth": [2,3,4,5],                   # Maksimum derinlik parametresi
             "n_estimators": [100,200,500,1000],       # Kullanılacak ağaç sayısını temsil etmektedir.
             "colsample_bytree": [0.4,0.7,1],          # Oluşturacak olan ağaçlardan oluşan alt küme oranını temsil etmektedir.
             }

In [15]:
# Şimdi yukarda oluşturduğum paremetreleri kullanarak işlem yapıyorum.
xgb_cv_model = GridSearchCV(xgb, xbg_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


In [16]:
# Şimid yapılan işlem sonrasında handi değerlerin daha başarılı çalıştığını anlamaya çalışıyorum.
xgb_cv_model.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.5,
 'max_depth': 5,
 'n_estimators': 1000}

In [17]:
xgb_tunned = XGBRFRegressor(colsample_bytree = 0.7,
                           learning_rate = 0.5,
                           max_depth = 2,
                           n_estimators = 1000).fit(X_train, y_train)

In [19]:
y_pred_1 = xgb_tunned.predict(X_test)

In [20]:
np.sqrt(mean_squared_error(y_test, y_pred_1))

395.50837693996914